## Exploratory Analysis



In [1]:
%load_ext autoreload
%autoreload 2
import os

from glob import glob
import pandas as pd

In [2]:
import csv
import pandas as pd

train_df = pd.read_table("../data/tass2020/task2/train.tsv", index_col=0, names=["id", "text", "label"], quoting=csv.QUOTE_NONE)

dev_df = pd.read_table(
    "../data/tass2020/task2/dev.tsv", index_col=0,names=["id", "text", "label"],
    quoting=csv.QUOTE_NONE)

test_df = pd.read_table("../data/tass2020/task2/test_task2.tsv", index_col=0,names=["id", "text", "label"], quoting=csv.QUOTE_NONE)
 

train_df.sample(10)

,text,label
id,,
860,"""Cuando te digan """"peleas como niña!"""", diles ...",others
967,HASHTAG | Alineación del Levante ante el Barce...,others
5235,Como se nota que HASHTAG ya no tiene el apoyo ...,disgust
2910,"""- Oiga vengo por lo de las elecciones, ¿Puedo...",others
2501,No me merece la pena irme a dormir ya HASHTAG,others
1136,¿Por qué es tan difícil comunicar el HASHTAG? ...,others
2333,"Ver a HASHTAG da tristeza, pero no más que la ...",sadness
5582,"""""""Nadie confrontaba con la ideología de géner...",others
1782,Vos estás loco Lionel HASHTAG. Nada más que tu...,others


One label per tweet

In [23]:
full_df = pd.read_table(
    "../data/tass2020/task2/dataset_emotions_SP.csv",
    names=["id", "text", "label", "offensive"], 
    index_col=0,
    skiprows=1,
)

print(full_df.shape)

full_df["slice"] = "train"

full_df[:3]

(8409, 3)


,text,label,offensive,slice
id,,,,
1,Acabo de ver la gran pérdida que estamos tenie...,sadness,0,train
2,USER ¿A que vamos a reconstruir Notre Dame ant...,sadness,0,train
3,Desde ayer andan sufriendo por el incendio de ...,anger,0,train


Bueno, hacemos un train test nuestro. Es un quilombo partirlo de otra manera

In [24]:
not_found = []

text2id = {}
for idx, row in full_df.iterrows():
    text2id[row.text] = idx


In [25]:
import re 
from tqdm.auto import tqdm
from fuzzywuzzy import fuzz

url_regex = re.compile(
    r"((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|co|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"
)


hashtag_regex = re.compile(r'\B#(\w*[a-zA-Z]+\w*)')

for idx, row in tqdm(test_df.iterrows()):
    text = url_regex.sub("URL", row.text)
    text = hashtag_regex.sub("HASHTAG", row.text)
    found = []
    for other, idx in text2id.items():
        if (ratio := fuzz.ratio(text, other)) >= 92:
            found.append((other, ratio))
            full_df.loc[idx, "slice"] = "test"
            
    if len(found) != 1:
        print("="*80)
        print(text)
        print(found)

tweet
[]
AHORA | Comunicado Grupo de Lima sobre la situación en HASHTAG. "Conminan a Maduro a cesar la usurpación", se declaran en sesión permanente, y deciden reunirse de manera presencial el próximo viernes 3 de mayo, en la ciudad de Lima, Perú.  URL
[('AHORA | Comunicado Grupo de Lima sobre la situación en HASHTAG. "Conminan a Maduro a cesar la usurpación", se declaran en sesión permanente, y deciden reunirse de manera presencial el próximo viernes 3 de mayo, en la ciudad de Lima, Perú.  URL', 100), ('AHORA | Aquí el Comunicado del Grupo de Lima sobre la situación en HASHTAG. "Conminan a Maduro a cesar la usurpación", se declaran en sesión permanente, y deciden reunirse de manera presencial el próximo viernes 3 de mayo, en la ciudad de Lima, Perú.  URL', 98)]
ATENCIÓN CONTENIDO VIOLENTO-Un vehículo de la Guardia Nacional arrolla a manifestantes que le tiraban piedras en HASHTAG, HASHTAG. USER USER  URL
[('ATENCIÓN CONTENIDO VIOLENTO-Un vehículo de la Guardia Nacional arrolla a manif

In [26]:
full_df["slice"].value_counts()

train    6732
test     1677
Name: slice, dtype: int64

In [28]:
test_df

,text,label
id,,
id,tweet,NaN
1,El 26 de octubre de 2018 grabé este vídeo dura...,NaN
2,Se incendió HASHTAG en USER pese a todas las m...,NaN
3,Me da tristeza saber que fue incendiado y toda...,NaN
4,Hoy es un día triste en HASHTAG HASHTAG catedr...,NaN
...,...,...
1661,Se han dado cuenta que todos esos mentirosos c...,NaN
1662,USER USER Pocas cosas son tan imbéciles como l...,NaN
1663,Es imposible describir lo que se vive en HASHT...,NaN


Vamos con esto...

In [29]:
new_train_df = full_df[full_df["slice"] == "train"].copy() 
new_test_df = full_df[full_df["slice"] == "test"].copy()

del new_train_df["slice"]
del new_test_df["slice"]

new_test_df["label"].value_counts(normalize=True), train_df["label"].value_counts(normalize=True)


(others      0.493143
 joy         0.215862
 sadness     0.118664
 anger       0.101371
 surprise    0.040549
 disgust     0.019678
 fear        0.010733
 Name: label, dtype: float64,
 others       0.490742
 joy          0.215730
 sadness      0.119925
 anger        0.101919
 surprise     0.040938
 disgust      0.019195
 fear         0.011381
 label        0.000170
 Name: label, dtype: float64)

In [30]:
new_train_df.to_csv("../data/emoevent/train_es.csv")
new_test_df.to_csv("../data/emoevent/test_es.csv")